# 01b_Data_Load_and_Preprocess

## Cel

Celem tego notebooka jest:

- Wczytanie i wstępna analiza surowych danych, identyfikacja braków i potencjalnych problemów.

- Obróbka brakujących wartości w zbiorach treningowym i testowym.

- Przygotowanie danych do modelowania poprzez:
    - Imputację brakujących wartości,
    - Standaryzację cech numerycznych,
    - Kodowanie zmiennych kategorycznych (Ordinal Encoding i One-Hot Encoding).

- Zapis przetworzonych danych w formacie CSV, gotowych do wykorzystania w modelach:
    - Regresji liniowej
    - Sieciach neuronowych.


## Kroki
**1. Wczytanie danych**

In [26]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

train_file_path = "../Data/Raw/train.csv"  
test_file_path = "../Data/Raw/test.csv"

train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)

In [27]:
selected_columns = [
    "OverallQual", "GrLivArea", "TotalBsmtSF", "GarageCars", "GarageArea",
    "1stFlrSF", "FullBath", "TotRmsAbvGrd", "YearBuilt", "YearRemodAdd",
    "Neighborhood", "MSZoning", "BldgType", "HouseStyle", "SaleCondition"
]

train_df = train_df[["Id"] + selected_columns + ["SalePrice"]]
test_df = test_df[["Id"] + selected_columns] 

**2. Analiza braków danych**

In [28]:
missing_train = train_df.isnull().sum()
missing_test = test_df.isnull().sum()

missing_train = missing_train[missing_train > 0]
missing_test = missing_test[missing_test > 0]

print("Brakujące dane w zbiorze treningowym:")
print(missing_train)

print("\nBrakujące dane w zbiorze testowym:")
print(missing_test)


Brakujące dane w zbiorze treningowym:
Series([], dtype: int64)

Brakujące dane w zbiorze testowym:
TotalBsmtSF    1
GarageCars     1
GarageArea     1
MSZoning       4
dtype: int64


**3. Imputacja brakujących danych**

In [29]:
impute_values = {
    "OverallQual": train_df["OverallQual"].median(),
    "GrLivArea": train_df["GrLivArea"].median(),
    "TotalBsmtSF": train_df["TotalBsmtSF"].median(),
    "GarageCars": train_df["GarageCars"].mode()[0],
    "GarageArea": train_df["GarageArea"].median(),
    "1stFlrSF": train_df["1stFlrSF"].median(),
    "FullBath": train_df["FullBath"].mode()[0],
    "TotRmsAbvGrd": train_df["TotRmsAbvGrd"].mode()[0],
    "YearBuilt": train_df["YearBuilt"].median(),
    "YearRemodAdd": train_df["YearRemodAdd"].median(),
    "Neighborhood": train_df["Neighborhood"].mode()[0],
    "MSZoning": train_df["MSZoning"].mode()[0],
    "BldgType": train_df["BldgType"].mode()[0],
    "HouseStyle": train_df["HouseStyle"].mode()[0],
    "SaleCondition": train_df["SaleCondition"].mode()[0]
}

train_df = train_df.fillna(impute_values)
test_df = test_df.fillna(impute_values)

print("\nBrakujące dane w zbiorze treningowym po imputacji:")
print(train_df.isnull().sum().sum())  
print("\nBrakujące dane w zbiorze testowym po imputacji:")
print(test_df.isnull().sum().sum()) 



Brakujące dane w zbiorze treningowym po imputacji:
0

Brakujące dane w zbiorze testowym po imputacji:
0


**4. Rozdzielenie kolumn na numeryczne i kategoryczne**


In [30]:
numerical_features = [
    "OverallQual", "GrLivArea", "TotalBsmtSF", "GarageCars", "GarageArea",
    "1stFlrSF", "FullBath", "TotRmsAbvGrd", "YearBuilt", "YearRemodAdd"
]

categorical_features = [
    "Neighborhood", "MSZoning", "BldgType", "HouseStyle", "SaleCondition"
]

ordinal_cols = ["BldgType", "HouseStyle", "SaleCondition"]
onehot_cols = ["Neighborhood", "MSZoning"]

ordinal_encoder = OrdinalEncoder()
train_df[ordinal_cols] = ordinal_encoder.fit_transform(train_df[ordinal_cols])
test_df[ordinal_cols] = ordinal_encoder.transform(test_df[ordinal_cols])  

onehot_encoder = OneHotEncoder(sparse_output=False, drop="first")  
train_onehot_encoded = onehot_encoder.fit_transform(train_df[onehot_cols])
test_onehot_encoded = onehot_encoder.transform(test_df[onehot_cols])  

train_onehot_df = pd.DataFrame(train_onehot_encoded, columns=onehot_encoder.get_feature_names_out(onehot_cols), index=train_df.index)
test_onehot_df = pd.DataFrame(test_onehot_encoded, columns=onehot_encoder.get_feature_names_out(onehot_cols), index=test_df.index)

train_df = train_df.drop(columns=onehot_cols).join(train_onehot_df)
test_df = test_df.drop(columns=onehot_cols).join(test_onehot_df)

**7. Skalowanie cech numerycznych do standardowego zakresu**

In [31]:
scaler = StandardScaler()
numerical_features_to_scale = [col for col in numerical_features if col != "Id"]

train_df[numerical_features_to_scale] = scaler.fit_transform(train_df[numerical_features_to_scale])
test_df[numerical_features_to_scale] = scaler.transform(test_df[numerical_features_to_scale])

**8. Zapisanie przetworzonych danych do plików CSV**


In [32]:
train_df.to_csv('../Data/Preprocessed/b_train_preprocessed.csv', index=False)
test_df.to_csv('../Data/Preprocessed/b_test_preprocessed.csv', index=False)

print("Przygotowane dane treningowe zapisano jako 'b_train_preprocessed.csv'.")
print("Przygotowane dane testowe zapisano jako 'b_test_preprocessed.csv'.")

Przygotowane dane treningowe zapisano jako 'b_train_preprocessed.csv'.
Przygotowane dane testowe zapisano jako 'b_test_preprocessed.csv'.
